In [1]:
import pandas as pd
import re

Part one

In [90]:
df = pd.read_excel("input_1.xlsx")

In [ ]:
total = 0 
for index, row in df.iterrows():
    string = df["schema"][index]
    numbers = re.compile(r'\b\d+\b').findall(string)
    matches = re.compile(r'\b\d+\b').finditer(string)
    indexes = [match.start() for match in matches]
    for quantity in range(0, len(indexes)):
        number_length = len(numbers[quantity])
        starting_index = indexes[quantity]
        if bool(re.compile(r'[^\w\s.]').search(string[max(starting_index - 1, 0):min(starting_index + number_length + 1, len(string) - 1)])) == True:
            total += int(numbers[quantity])
            continue
        if bool(re.compile(r'[^\w\s.]').search(df["schema"][max(index - 1, 0)][max(starting_index - 1, 0):min(starting_index + number_length + 1, len(df["schema"][max(index - 1, 0)]) - 1)])) == True:
            total += int(numbers[quantity])
            continue
        if bool(re.compile(r'[^\w\s.]').search(df["schema"][min(index + 1, len(df) - 1)][max(starting_index - 1, 0):min(starting_index + number_length + 1, len(df["schema"][min(index + 1, len(df) - 1)]) - 1)])) == True:
            total += int(numbers[quantity]) 
            continue

In [ ]:
total

Part two

In [203]:
total = 0 
numbers_near_stars = pd.DataFrame(columns = ['number', 'star_line', 'star_index'])
for index, row in df.iterrows():
    string = df["schema"][index]
    numbers = re.compile(r'\b\d+\b').findall(string)
    matches = re.compile(r'\b\d+\b').finditer(string)
    indexes = [match.start() for match in matches]
    for quantity in range(0, len(indexes)):
        number_length = len(numbers[quantity])
        starting_index = indexes[quantity]
        if bool(re.compile(r'\*').search(string[max(starting_index - 1, 0):min(starting_index + number_length + 1, len(string) - 1)])) == True:
            star_index = string[max(starting_index - 1, 0):min(starting_index + number_length + 1, len(string) - 1)].find('*') + max(starting_index - 1, 0)
            numbers_near_stars = pd.concat([numbers_near_stars,  pd.DataFrame({'number': numbers[quantity], 'star_line': [index], 'star_index': [star_index]})], ignore_index=True )
            continue
        if bool(re.compile(r'\*').search(df["schema"][max(index - 1, 0)][max(starting_index - 1, 0):min(starting_index + number_length + 1, len(df["schema"][max(index - 1, 0)]) - 1)])) == True:
            star_index = df["schema"][max(index - 1, 0)][max(starting_index - 1, 0):min(starting_index + number_length + 1, len(df["schema"][max(index - 1, 0)]) - 1)].find('*') + max(starting_index - 1, 0)
            numbers_near_stars = pd.concat([numbers_near_stars,  pd.DataFrame({'number': numbers[quantity], 'star_line': [index - 1], 'star_index': [star_index]})], ignore_index=True )
            continue
        if bool(re.compile(r'\*').search(df["schema"][min(index + 1, len(df) - 1)][max(starting_index - 1, 0):min(starting_index + number_length + 1, len(df["schema"][min(index + 1, len(df) - 1)]) - 1)])) == True:
            star_index = df["schema"][min(index + 1, len(df) - 1)][max(starting_index - 1, 0):min(starting_index + number_length + 1, len(df["schema"][min(index + 1, len(df) - 1)]) - 1)].find('*') + max(starting_index - 1, 0)
            numbers_near_stars = pd.concat([numbers_near_stars,  pd.DataFrame({'number': numbers[quantity], 'star_line': [index + 1], 'star_index': [star_index]})], ignore_index=True )
            continue

In [204]:
gears = numbers_near_stars.groupby(['star_line', 'star_index']).size().reset_index(name='count')
gears = gears[gears['count'] == 2].drop(columns='count')
gears = pd.merge(gears, numbers_near_stars, on=['star_line', 'star_index'], how='left')
gears['number'] = pd.to_numeric(gears['number']) 
gears = gears.groupby(['star_line', 'star_index'])['number'].apply(lambda x: int(x.prod())).reset_index()
sum(gears['number'])